<a href="https://colab.research.google.com/github/filipespc/sus-kpis-analysis/blob/main/sia/tbl_exames_rastreamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações iniciais para conectar com bucket no Google Storage
 - Autenticação do Google Colab
 - Definição do nome do projeto

In [ ]:
from google.colab import auth
auth.authenticate_user()

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

!gcloud config set project {project_id}

Are you sure you wish to set property [core/project] to teak-ellipse-317015?

Do you want to continue (Y/n)?  Y

Updated property [core/project].


# Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  76878      0 --:--:-- --:--:-- --:--:-- 74617
OK
64 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 64 not upgraded.
Need to get 11.6 MB of archives.
After this operation, 27.4 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.4_amd64.deb ...
Unpacking gcsfuse (0.41.4) ...
Setting up gcsfuse (0.41.4) ...


In [ ]:
serice_account_json = '/content/gcp-leitura.json'

# Montagem do bucket em uma pasta local do google colab

In [ ]:
!mkdir {local_folder_name}
!gcsfuse --key-file {serice_account_json} --implicit-dirs {bucket_id} {local_folder_name}

2022/07/17 14:31:54.924889 Start gcsfuse/0.41.4 (Go version go1.17.6) for app "" using mount point: /content/colab
2022/07/17 14:31:54.940610 Opening GCS connection...
2022/07/17 14:31:55.354217 Mounting file system "observatorio-oncologia"...
2022/07/17 14:31:55.386336 File system has been successfully mounted.


In [ ]:
import glob
import os
import pandas as pd

def get_files(state, year, month, file_type, file_group):
  initial_path = os.path.join(r'/content/',local_folder_name,project_folder_name)
  internal_folder = f"""{state}/{year}/{month}/{file_type}/{file_group}"""
  # print(f"{initial_path}/{internal_folder}/*.parquet.gzip")
  return glob.glob(f"{initial_path}/{internal_folder}/*.parquet.gzip")

def read_files(state, year, month, file_type, file_group):
  file_paths = get_files(state, year, month, file_type, file_group)
  return pd.concat([pd.read_parquet(file_path) for file_path in file_paths])

In [ ]:
df = read_files('SP','2022','02','SIA','PA')

with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
  display(df.head(10))

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,PA_CNPJCPF,PA_CNPJMNT,PA_CNPJ_CC,PA_MVM,PA_CMP,PA_PROC_ID,PA_TPFIN,PA_SUBFIN,PA_NIVCPL,PA_DOCORIG,PA_AUTORIZ,PA_CNSMED,PA_CBOCOD,PA_MOTSAI,PA_OBITO,PA_ENCERR,PA_PERMAN,PA_ALTA,PA_TRANSF,PA_CIDPRI,PA_CIDSEC,PA_CIDCAS,PA_CATEND,PA_IDADE,IDADEMIN,IDADEMAX,PA_FLIDADE,PA_SEXO,PA_RACACOR,PA_MUNPCN,PA_QTDPRO,PA_QTDAPR,PA_VALPRO,PA_VALAPR,PA_UFDIF,PA_MNDIF,PA_DIF_VAL,NU_VPA_TOT,NU_PA_TOT,PA_INDICA,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016296545539,223415,00,0,0,0,0,0,0000,0000,0000,01,029,0,130,1,M,99,354140,1,1,0.0,0.0,0,0,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
1,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016277909539,221205,00,0,0,0,0,0,0000,0000,0000,01,041,0,130,1,F,99,354140,1,1,0.0,0.0,0,0,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
2,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016277909539,221205,00,0,0,0,0,0,0000,0000,0000,01,062,0,130,1,F,99,352060,1,1,0.0,0.0,0,1,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
3,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,709804029406697,221105,00,0,0,0,0,0,0000,0000,0000,01,072,0,130,1,F,99,350130,1,1,0.0,0.0,0,1,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
4,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,123233468320004,221105,00,0,0,0,0,0,0000,0000,0000,01,069,0,130,1,M,99,350130,1,1,0.0,0.0,0,1,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
5,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016277909539,221205,00,0,0,0,0,0,0000,0000,0000,01,042,0,130,1,M,99,355430,1,1,0.0,0.0,0,1,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
6,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016294344625,221205,00,0,0,0,0,0,0000,0000,0000,01,023,0,130,1,F,99,354140,1,1,0.0,0.0,0,0,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
7,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,123233468320004,221105,00,0,0,0,0,0,0000,0000,0000,01,085,0,130,1,F,99,354140,1,1,0.0,0.0,0,0,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
8,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016277909539,221205,00,0,0,0,0,0,0000,0000,0000,01,052,0,130,1,F,99,354140,1,1,0.0,0.0,0,0,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023
9,2750562,350000,EP,354140,7101,0000,0000,80,00,M,46374500000194,46374500000194,00000000000000,202202,202202,0213010720,07,0000,2,I,0000000000000,980016296545539,223415,00,0,0,0,0,0,0000,0000,0000,01,036,0,130,1,M,99,353920,1,1,0.0,0.0,0,1,0.0,0.0,0.0,5,1,K,0,,0.0,0.0,0.0,145003,,1023


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
procedimentos_layout = pd.read_csv('/content/drive/MyDrive/Datacare - CA mama/arquivos_auxiliares/SigTap/202003/tb_procedimento_layout.txt')
procedimentos = pd.read_csv('/content/drive/MyDrive/Datacare - CA mama/arquivos_auxiliares/SigTap/202003/tb_procedimento.txt', names=['all_info'], encoding='latin-1', sep=';')

for idx, col_info in procedimentos_layout.iterrows():
  if col_info.Tipo == 'NUMBER':
    procedimentos[col_info.Coluna] = procedimentos.all_info.str[col_info.Inicio-1:col_info.Fim].astype(int)
    if col_info.Coluna.startswith('VL_'):
      procedimentos[col_info.Coluna] = procedimentos[col_info.Coluna]/100
  else:
    procedimentos[col_info.Coluna] = procedimentos.all_info.str[col_info.Inicio-1:col_info.Fim]

procedimentos.drop(columns=['all_info'], inplace=True)

In [ ]:
for id, row in procedimentos.loc[procedimentos.NO_PROCEDIMENTO.str.contains("MAMOGRAFIA"), ['CO_PROCEDIMENTO','NO_PROCEDIMENTO']].iterrows():
  print('Número de ' + row.NO_PROCEDIMENTO.strip() + ' (' + row.CO_PROCEDIMENTO.strip() + ')' + ' relizadas:', end='')
  print(len(df[df.PA_PROC_ID == row.CO_PROCEDIMENTO]))

Número de MAMOGRAFIA (0204030030) relizadas:7035
Número de MARCACAO PRE-CIRURGICA DE LESAO NAO PALPAVEL DE MAMA ASSOCIADA A MAMOGRAFIA (0204030048) relizadas:148
Número de MAMOGRAFIA BILATERAL PARA RASTREAMENTO (0204030188) relizadas:83165


In [ ]:
for id, row in procedimentos.loc[procedimentos.NO_PROCEDIMENTO.str.contains("ULTRASSONOGRAFIA MAMARIA"), ['CO_PROCEDIMENTO','NO_PROCEDIMENTO']].iterrows():
  print('Número de ' + row.NO_PROCEDIMENTO.strip() + ' (' + row.CO_PROCEDIMENTO.strip() + ')' + ' relizadas:', end='')
  print(len(df[df.PA_PROC_ID == row.CO_PROCEDIMENTO]))

Número de ULTRASSONOGRAFIA MAMARIA BILATERAL (0205020097) relizadas:44309


In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
  display(df[df.PA_PROC_ID == "0204030030"])

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,PA_CNPJCPF,PA_CNPJMNT,PA_CNPJ_CC,PA_MVM,PA_CMP,PA_PROC_ID,PA_TPFIN,PA_SUBFIN,PA_NIVCPL,PA_DOCORIG,PA_AUTORIZ,PA_CNSMED,PA_CBOCOD,PA_MOTSAI,PA_OBITO,PA_ENCERR,PA_PERMAN,PA_ALTA,PA_TRANSF,PA_CIDPRI,PA_CIDSEC,PA_CIDCAS,PA_CATEND,PA_IDADE,IDADEMIN,IDADEMAX,PA_FLIDADE,PA_SEXO,PA_RACACOR,PA_MUNPCN,PA_QTDPRO,PA_QTDAPR,PA_VALPRO,PA_VALAPR,PA_UFDIF,PA_MNDIF,PA_DIF_VAL,NU_VPA_TOT,NU_PA_TOT,PA_INDICA,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
3699,2083086,350000,EP,352530,7101,0000,0000,07,00,I,50753755000135,00000000000000,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,210160104960018,225320,00,0,0,0,0,0,0000,0000,0000,01,051,10,130,1,F,01,350400,2,2,45.0,45.0,0,1,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,3069
3725,2083086,350000,EP,352530,7101,0000,0000,07,00,I,50753755000135,00000000000000,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,210162166390004,225320,00,0,0,0,0,0,0000,0000,0000,01,064,10,130,1,F,01,350790,1,1,22.5,22.5,0,1,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,3069
3881,2083086,350000,EP,352530,7101,0000,0000,07,00,I,50753755000135,00000000000000,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,210160104960018,225320,00,0,0,0,0,0,0000,0000,0000,01,070,10,130,1,F,03,352390,2,2,45.0,45.0,0,1,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,3069
3882,2083086,350000,EP,352530,7101,0000,0000,07,00,I,50753755000135,00000000000000,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,210160104960018,225320,00,0,0,0,0,0,0000,0000,0000,01,058,10,130,1,F,01,352000,2,2,45.0,45.0,0,1,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,3069
5219,6432530,350000,EP,355030,7101,0000,0000,04,00,M,46374500017402,46374500000194,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,980016294068725,225320,00,0,0,0,0,0,0000,0000,0000,01,042,10,130,1,F,03,355030,2,2,45.0,45.0,0,0,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,1023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494634,2751933,355030,PG,355030,0000,0000,0000,62,00,M,46392130000380,46392130000380,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,980016278532358,225320,00,0,0,0,0,0,0000,0000,0000,01,059,10,130,1,F,03,355030,2,2,45.0,45.0,0,0,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,1031
1494855,2751933,355030,PG,355030,0000,0000,0000,62,00,M,46392130000380,46392130000380,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,980016278532358,225320,00,0,0,0,0,0,0000,0000,0000,01,070,10,130,1,F,99,355030,2,2,45.0,45.0,0,0,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,1031
1494858,2751968,355030,PG,355030,0000,0000,0000,04,00,M,46392130000380,46392130000380,00000000000000,202202,202202,0204030030,06,0000,2,I,0000000000000,980016003021336,225320,00,0,0,0,0,0,0000,0000,0000,01,059,10,130,1,F,99,355030,1,1,22.5,22.5,0,0,0.0,0.0,22.5,5,1,K,0,,0.0,0.0,0.0,121012,,1031
1496456,2751933,355030,PG,355030,0000,0000,0000,62,00,M,46392130000380,46392130000380,00000000000000,202202,202201,0204030030,06,0000,2,I,0000000000000,980016278532358,225320,00,0,0,0,0,0,0000,0000,0000,01,058,10,130,1,F,01,355030,2,2,45.0,45.0,0,0,0.0,0.0,22.5,5,1,S,0,,0.0,0.0,0.0,121012,,1031
